In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
#from keras.optimizers import SGD, Adadelta, Adagrad, RMSprop, Adam
from tensorflow.keras.optimizers import SGD ,Adadelta, Adagrad,RMSprop, Adam

from sklearn import svm

from six.moves import cPickle as pickle

In [2]:
pickle_files = [r'C:\Users\chida\OneDrive\Desktop\back\Driver Drowsiness Monitoring using\Driver Drowsiness Monitoring using\M2\yawn_detection/yawn_mouths.pickle']
i = 0
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        if i == 0:
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
        else:
            print("here")
            train_dataset = np.concatenate((train_dataset, save['train_dataset']))
            train_labels = np.concatenate((train_labels, save['train_labels']))
            test_dataset = np.concatenate((test_dataset, save['test_dataset']))
            test_labels = np.concatenate((test_labels, save['test_labels']))
        del save  # hint to help gc free up memory
    i += 1

In [3]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (59, 60, 60, 1) (59, 1)
Test set (15, 60, 60, 1) (15, 1)


In [4]:
batch_size = 1
nb_classes = 1
epochs = 20

In [5]:

X_train = train_dataset
X_train = X_train.reshape(59,60,60,1)
# X_train = X_train.reshape(X_train.shape[0:3])
# X_train = X_train.reshape(len(X_train), -1)
Y_train = train_labels

X_test = test_dataset
X_test = X_test.reshape(15,60,60,1)
# X_test = X_test.reshape(X_test.shape[0:3])
# X_test = X_test.reshape(len(X_test), -1)
Y_test = test_labels

In [6]:
# input image dimensions
_, img_channels, img_rows, img_cols = X_train.shape

# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
model = Sequential()

model.add(Convolution2D(32, (3, 3), padding='same',
                        input_shape=(60,60,1),data_format='channels_last'))
model.add(Activation('relu'))
model.add(Convolution2D(24, (3, 3), data_format='channels_last'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), padding='same', data_format='channels_last'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

In [8]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0005), metrics=['accuracy'])

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [9]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=[X_test, Y_test])

Epoch 1/20
59/59 - 4s - loss: 0.7288 - accuracy: 0.5085 - val_loss: 0.6758 - val_accuracy: 0.5333 - 4s/epoch - 71ms/step
Epoch 2/20
59/59 - 3s - loss: 0.6917 - accuracy: 0.6780 - val_loss: 0.6115 - val_accuracy: 0.6000 - 3s/epoch - 57ms/step
Epoch 3/20
59/59 - 3s - loss: 0.4169 - accuracy: 0.8305 - val_loss: 0.2471 - val_accuracy: 0.9333 - 3s/epoch - 55ms/step
Epoch 4/20
59/59 - 3s - loss: 0.4582 - accuracy: 0.8983 - val_loss: 0.3455 - val_accuracy: 0.8667 - 3s/epoch - 51ms/step
Epoch 5/20
59/59 - 3s - loss: 0.2145 - accuracy: 0.8814 - val_loss: 0.2906 - val_accuracy: 0.9333 - 3s/epoch - 50ms/step
Epoch 6/20
59/59 - 3s - loss: 0.1407 - accuracy: 0.9661 - val_loss: 0.3015 - val_accuracy: 0.9333 - 3s/epoch - 56ms/step
Epoch 7/20
59/59 - 3s - loss: 0.0839 - accuracy: 0.9661 - val_loss: 0.2275 - val_accuracy: 0.9333 - 3s/epoch - 52ms/step
Epoch 8/20
59/59 - 3s - loss: 0.1758 - accuracy: 0.9661 - val_loss: 0.1109 - val_accuracy: 0.9333 - 3s/epoch - 56ms/step
Epoch 9/20
59/59 - 4s - loss: 0.

In [10]:
score = model.evaluate(X_test, Y_test,  verbose=1)

1/1 [==============================] - 0s 118ms/step - loss: 0.7763 - accuracy: 0.8667


In [11]:

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7763091921806335
Test accuracy: 0.8666666746139526
